In [2]:
# -*- coding: utf-8 -*-
from matplotlib import pyplot as plt
from tqdm import tqdm
import numpy as np
import copy
import json
import os
import torch.optim as optim

from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
import torch.nn as nn
import torch


from src.data.argoverse_datamodule import ArgoverseDataModule
from src.data.argoverse_dataset import ArgoverseDataset
from src.data.dummy_datamodule import DummyDataModule
from src.models.WIMP import WIMP
import math

import torch.backends.cudnn as cudnn
import random
os.environ["CUDA_VISIBLE_DEVICES"]= "1"

In [71]:
args = {"IFC":True, "add_centerline":False, "attention_heads":4, "batch_norm":False, "batch_size":100, "check_val_every_n_epoch":3, 
          "dataroot":'./data/LRP_adjacency', "dataset":'argoverse', "distributed_backend":'ddp', "dropout":0.0, 
          "early_stop_threshold":5, "experiment_name":'example', "gpus":3, "gradient_clipping":True, "graph_iter":1, 
          "hidden_dim":512, "hidden_key_generator":True, "hidden_transform":False, "input_dim":2, "k_value_threshold":10, 
          "k_values":[6, 5, 4, 3, 2, 1], "lr":0.0001, "map_features":False, "max_epochs":200, "mode":'train', "model_name":'WIMP', 
          "no_heuristic":False, "non_linearity":'relu', "num_layers":4, "num_mixtures":6, "num_nodes":1, "output_conv":True, "output_dim":2, 
          "output_prediction":True, "precision":32, "predict_delta":False, "resume_from_checkpoint":None, 
          "scheduler_step_size":[60, 90, 120, 150, 180], "seed":None, "segment_CL":False, "segment_CL_Encoder":False, 
          "segment_CL_Encoder_Gaussian":False, "segment_CL_Encoder_Gaussian_Prob":False, "segment_CL_Encoder_Prob":True, 
          "segment_CL_Gaussian_Prob":False, "segment_CL_Prob":False, "use_centerline_features":True, "use_oracle":False, "waypoint_step":5, 
          "weight_decay":0.0, "workers":8, "wta":False, "draw_image" : False, "remove_high_related_score" : False, "maximum_delete_num" : 3, 
          "save_json": False, "make_submit_file" : False, "use_hidden_feature" : True, "is_LRP": False, "adjacency_exp" : True}

from argparse import ArgumentParser
parser = ArgumentParser()


for k in args:
    parser.add_argument(str("--" + k), default = args[k], type= type(args[k]))
parser.add_argument("--XAI_lambda", default = 0.2, type= float)
parser.add_argument("--name", default = "", type=str)

parser = parser.parse_args(args=[])

In [72]:
def get_metric(metric_dict, ade,fde,mr,loss, length):
    metric_dict["ade"] += (ade * length).cpu().item()
    metric_dict["fde"] += (fde * length).cpu().item()
    metric_dict["mr"] += (mr * length).cpu().item()
    metric_dict["loss"] += (loss * length).cpu().item()
    metric_dict["length"]+=length

def calc_mean(metric):
    metric["fde"] /= metric["length"]
    metric["ade"] /= metric["length"]
    metric["mr"] /= metric["length"]
    metric["loss"] /= metric["length"]
    return {
        "fde": metric["fde"],
        "ade": metric["ade"],
        "loss": metric["loss"],
        "mr": metric["mr"]
    }

Relu = nn.ReLU()
softmax = torch.nn.Softmax(dim=2)


def normalize_max1(w):
    for i in range(len(w)):
        w[i] = w[i] / torch.max(abs(w[i]))
    return w


# f: adjacency matrix를 어떻게 normalize할 지(현재는 lrp값으로 되어 있음)
def Test(dataset, f, f_name=None):
    torch.manual_seed(0)
    torch.cuda.manual_seed(0)
    torch.cuda.manual_seed_all(0)
    np.random.seed(0)
    cudnn.benchmark = False
    cudnn.deterministic = True
    random.seed(0)
    torch.backends.cudnn.benchmark = False

    metrics = {
        "ade": 0.0,
        "fde": 0.0,
        "mr": 0.0,
        "loss": 0.0,
        "length": 0,
    }
    
    for batch_idx, batch in enumerate(tqdm(dataset)):
        input_dict, target_dict = batch[0], batch[1]

        # get cuda
        input_dict["agent_features"] = input_dict["agent_features"].cuda()
        input_dict["social_features"] = input_dict["social_features"].cuda()
        input_dict["social_label_features"] = input_dict["social_label_features"].cuda()
        input_dict["label_adjacency"] = input_dict["label_adjacency"].cuda()
        input_dict["num_agent_mask"] = input_dict["num_agent_mask"].cuda()
        input_dict["ifc_helpers"]["agent_oracle_centerline"] = input_dict["ifc_helpers"]["agent_oracle_centerline"].cuda()
        input_dict["ifc_helpers"]["agent_oracle_centerline_lengths"] = input_dict["ifc_helpers"]["agent_oracle_centerline_lengths"].cuda()
        input_dict["ifc_helpers"]["social_oracle_centerline"] = input_dict["ifc_helpers"]["social_oracle_centerline"].cuda()
        input_dict["ifc_helpers"]["social_oracle_centerline_lengths"] = input_dict["ifc_helpers"]["social_oracle_centerline_lengths"].cuda()
        input_dict["ifc_helpers"]["agent_oracle_centerline"] = input_dict["ifc_helpers"]["agent_oracle_centerline"].cuda()
        input_dict["adjacency"] = input_dict["adjacency"].cuda()
        
        target_dict["agent_labels"] = target_dict["agent_labels"].cuda()
        input_dict["adjacency"] =f(input_dict["adjacency"], input_dict['num_agent_mask'])
        
        input_dict['adjacency'][:,0,0] = input_dict['adjacency'][:,0,0].clamp(min = 1.0)

        with torch.no_grad():
            preds, waypoint_preds, all_dist_params, attention, adjacency, gan_features, graph_output = model(**input_dict)

            loss, (ade, fde, mr) = model.eval_preds(preds, target_dict, waypoint_preds)

            get_metric(metrics, ade, fde, mr, loss, len(input_dict["adjacency"]))

    write_json = {
        "metric": calc_mean(metrics)
    }
    print(f_name)
    print(write_json)


In [73]:
train_loader = ArgoverseDataset(parser.dataroot, mode='train', delta=parser.predict_delta,
                              map_features_flag=parser.map_features,
                              social_features_flag=True, heuristic=(not parser.no_heuristic),
                              ifc=parser.IFC, is_oracle=parser.use_oracle)

val_loader = ArgoverseDataset(parser.dataroot, mode='val', delta=parser.predict_delta,
                              map_features_flag=parser.map_features,
                              social_features_flag=True, heuristic=(not parser.no_heuristic),
                              ifc=parser.IFC, is_oracle=parser.use_oracle)

train_dataset = DataLoader(train_loader, batch_size=parser.batch_size, num_workers=parser.workers,
                                pin_memory=True, collate_fn=ArgoverseDataset.collate,
                                shuffle=True, drop_last=True)

val_dataset = DataLoader(val_loader, batch_size=parser.batch_size, num_workers=parser.workers,
                                pin_memory=True, collate_fn=ArgoverseDataset.collate,
                                shuffle=False, drop_last=False)

model = WIMP(parser)
model.load_state_dict(torch.load("experiments/example_old/checkpoints/epoch=122.ckpt")['state_dict'], strict=False) # 학습할 때에는 graph 모듈에서 p에 해당하는 network가 없었으므로

model = model.cuda()
optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)


In [74]:
softmax = torch.nn.Softmax(dim=2)

f1 = lambda w, _: softmax(normalize_max1(abs(w))) * len(w[0])
# f2 = lambda w, _: softmax(normalize_max1(relu(w) + 0.000001)) * len(w[0])
f3 = lambda w, _: softmax(normalize_max1(abs(w))*2) * len(w[0])
# f4 = lambda w, _: softmax(normalize_max1(relu(w))*2 + 0.000001) * len(w[0])
f5 = lambda w, _: softmax(normalize_max1(abs(w))*0.5) * len(w[0])
# f6 = lambda w, _: softmax(normalize_max1(relu(w))*0.5 + 0.000001) * len(w[0])

f10 = lambda w, _: softmax(abs(w)) * len(w[0])
f11 = lambda w, _: softmax(abs(w)*0.5) * len(w[0])
f12 = lambda w, _: softmax(abs(w)*2) * len(w[0])

for f in [f3, f5, f1]:
    Test(val_dataset, f, "softmax(normalize_max1(abs(w))) * len(w[0])")

  6%|▋         | 25/395 [00:07<01:55,  3.19it/s]


KeyboardInterrupt: 

In [58]:
for f in [f3, f5]:
    Test(val_dataset, f, "softmax(normalize_max1(abs(w))) * len(w[0])")

  2%|▏         | 6/395 [00:02<03:09,  2.05it/s]


KeyboardInterrupt: 

In [61]:
softmax = torch.nn.Softmax(dim=0)

def f1(w, mask):
    w = abs(w)
    for i in range(len(w)):
#         print(w[i].shape)
        num_agent = int(torch.sum(mask[i]))+1
#         print(num_agent)
        w[i,0,:num_agent] = softmax(w[i,0,:num_agent]*0.25) * num_agent
        w[i,1:,:] = 1
    return w
    
# f1 = lambda w: softmax(normalize_max1(abs(w))) * len(w[0])
Test(val_dataset, f1, "softmax(normalize_max1(abs(w))) * len(w[0])")

100%|██████████| 395/395 [01:45<00:00,  3.74it/s]

softmax(normalize_max1(abs(w))) * len(w[0])
{'metric': {'fde': 1.1460760357798379, 'ade': 0.7543620654432021, 'loss': 6.051380298956232, 'mr': 0.11765301784475564}}


In [70]:
f1 = lambda w: softmax(normalize_max1(abs(w))*2) * len(w[0])
to_gaussian = lambda arr, mean = 0, std = 1: ((arr - torch.mean(arr))/ (torch.std(arr) + 0.00001)) * std + mean
softmax0 = torch.nn.Softmax(dim=0)


def f2(w, mask = None):
    for i in range(len(w)):
        num_agent = int(torch.sum(mask[i]))+1
        
        w[i,0, :num_agent] = (softmax0(to_gaussian(abs(w[i,0, :num_agent]) * num_agent, mean = 1, std = 0.12))*num_agent) ** 1.5
#         print(w[i,0, :num_agent])
#         assert False, "SS"
    return w

Test(val_dataset, f2, "to_gaussian(w[0,0, :num_agent], mean = 1, std = 0.1)")

100%|██████████| 395/395 [01:47<00:00,  3.66it/s]

to_gaussian(w[0,0, :num_agent], mean = 1, std = 0.1)
{'metric': {'fde': 1.144936785796513, 'ade': 0.7533492711472715, 'loss': 6.033516781408389, 'mr': 0.11674097837539738}}


In [78]:
f1 = lambda w: softmax(normalize_max1(abs(w))*2) * len(w[0])
to_gaussian = lambda arr, mean = 0, std = 1: ((arr - torch.mean(arr))/ (torch.std(arr) + 0.00001)) * std + mean
softmax0 = torch.nn.Softmax(dim=0)


for std in range(10, 20):
    print(std/100)
    def f2(w, mask = None):
        for i in range(len(w)):
            num_agent = int(torch.sum(mask[i]))+1
            w[i,0, :num_agent] = softmax0(to_gaussian(abs(w[i,0, :num_agent]) * num_agent, mean = 1, std = std/100))*num_agent
        return w



    Test(val_dataset, f2, "to_gaussian(w[0,0, :num_agent], mean = 1, std = 0.1)")


  0%|          | 0/395 [00:00<?, ?it/s]

0.1


  0%|          | 0/395 [00:00<?, ?it/s]

to_gaussian(w[0,0, :num_agent], mean = 1, std = 0.1)
{'metric': {'fde': 1.1439233589713564, 'ade': 0.7533491165182781, 'loss': 6.0448233631877795, 'mr': 0.1165636378258503}}
0.11


  0%|          | 0/395 [00:00<?, ?it/s]

...

to_gaussian(w[0,0, :num_agent], mean = 1, std = 0.1)
{'metric': {'fde': 1.1448083592659557, 'ade': 0.7537462764487945, 'loss': 6.043074806160805, 'mr': 0.11620895586904843}}
0.18


  0%|          | 0/395 [00:00<?, ?it/s]

to_gaussian(w[0,0, :num_agent], mean = 1, std = 0.1)
{'metric': {'fde': 1.1453997102110407, 'ade': 0.7541039863674565, 'loss': 6.043130600167326, 'mr': 0.11623429047660302}}
0.19


100%|██████████| 395/395 [01:48<00:00,  3.64it/s]

to_gaussian(w[0,0, :num_agent], mean = 1, std = 0.1)
{'metric': {'fde': 1.1459321774690316, 'ade': 0.7545272774381293, 'loss': 6.043216008718651, 'mr': 0.11653830341158196}}


In [35]:
from matplotlib import pyplot as plt

for ii, d in enumerate(val_dataset):
    
    input_dict, traget_dict = d[0], d[1]
    break
# plt.show()
a = torch.tensor([[1,2,3],[1,2,3]]).float()
print(input_dict['adjacency'][0,0])
print(f1(input_dict['adjacency'], input_dict['num_agent_mask'])[0,0])


tensor([ 0.0421,  0.0122,  0.0135, -0.0362,  0.0171, -0.0192, -0.0164,  0.0150,
         0.0025,  0.0181, -0.0038, -0.0078,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000])
tensor([1.1896, 1.0306, 1.0422, 1.1675, 1.0692, 1.0836, 1.0642, 1.0540, 0.8833,
        1.0761, 0.9177, 0.9842, 0.4376, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000])


In [24]:
f1 = lambda w: softmax(normalize_max1(abs(w))*2) * len(w[0])
to_gaussian = lambda arr, mean = 0, std = 1: ((arr - torch.mean(arr))/ (torch.std(arr) + 0.00001)) * std + mean


def f2(w, mask = None):
    for i in range(len(w)):
        num_agent = int(torch.sum(mask[i]))
        
        w[i,0, :num_agent] = to_gaussian(abs(w[i,0, :num_agent]), mean = 1, std = 0.1)
    return w

Test(val_dataset, f2, "to_gaussian(w[0,0, :num_agent], mean = 1, std = 0.1)")

100%|██████████| 395/395 [01:46<00:00,  3.70it/s]

to_gaussian(w[0,0, :num_agent], mean = 1, std = 0.1)
{'metric': {'fde': 1.1442772898540643, 'ade': 0.7533639042687696, 'loss': 6.041107798569783, 'mr': 0.11671564417857606}}


In [37]:
f1 = lambda w: softmax(normalize_max1(abs(w))*2) * len(w[0])
to_gaussian = lambda arr, mean = 0, std = 1: ((arr - torch.mean(arr))/ (torch.std(arr) + 0.00001)) * std + mean
softmax0 = torch.nn.Softmax(dim=0)


def f2(w, mask = None):
    for i in range(len(w)):
        num_agent = int(torch.sum(mask[i]))+1
        
        w[i,0, :num_agent] = to_gaussian(softmax0(abs(w[i,0, :num_agent])) * num_agent, mean = 1, std = 0.12)
        print(w[i,0, :num_agent])
        assert False, "SS"
    return w

Test(val_dataset, f2, "to_gaussian(w[0,0, :num_agent], mean = 1, std = 0.1)")

100%|██████████| 395/395 [01:45<00:00,  3.73it/s]

to_gaussian(w[0,0, :num_agent], mean = 1, std = 0.1)
{'metric': {'fde': 1.144170692878355, 'ade': 0.7532816630990872, 'loss': 6.039364519803496, 'mr': 0.11653830343574273}}


In [ ]:
f1 = lambda w: softmax(normalize_max1(abs(w))*2) * len(w[0])
to_gaussian = lambda arr, mean = 0, std = 1: ((arr - torch.mean(arr))/ (torch.std(arr) + 0.00001)) * std + mean


for std in range(1, 100):
    def f2(w, mask = None, mean = 1, std = std/100):
        for i in range(len(w)):
            num_agent = int(torch.sum(mask[i]))

            w[i,0, :num_agent] = to_gaussian(abs(w[i,0, :num_agent]), mean = mean, std = std)
        return w

    Test(val_dataset, f2, f"to_gaussian(w[0,0, :num_agent], mean = 1, std = {std/100})")

  0%|          | 0/395 [00:00<?, ?it/s]

to_gaussian(w[0,0, :num_agent], mean = 1, std = 0.01)
{'metric': {'fde': 1.1459254470488105, 'ade': 0.7541138015394516, 'loss': 6.050708923834657, 'mr': 0.11777968962616799}}


  0%|          | 0/395 [00:00<?, ?it/s]

to_gaussian(w[0,0, :num_agent], mean = 1, std = 0.02)
{'metric': {'fde': 1.1457067941140684, 'ade': 0.7540020617000671, 'loss': 6.049396532674912, 'mr': 0.11793169630506428}}


  0%|          | 0/395 [00:00<?, ?it/s]

...

  0%|          | 0/395 [00:00<?, ?it/s]

to_gaussian(w[0,0, :num_agent], mean = 1, std = 0.6)
{'metric': {'fde': 1.1519634454029022, 'ade': 0.75705187920863, 'loss': 6.030067751559353, 'mr': 0.11861572541782718}}


  0%|          | 0/395 [00:00<?, ?it/s]

to_gaussian(w[0,0, :num_agent], mean = 1, std = 0.61)
{'metric': {'fde': 1.1523433066013615, 'ade': 0.7573017387869486, 'loss': 6.030222504255884, 'mr': 0.11886906953634982}}


 87%|████████▋ | 345/395 [01:33<00:13,  3.84it/s]

In [16]:
from matplotlib import pyplot as plt

for ii, d in enumerate(val_dataset):
    
    input_dict, traget_dict = d[0], d[1]
#     for i in range(100):
#         xy = d[0]['agent_features'][i]
#         xy.shape
#         plt.scatter(xy[:,0], xy[:, 1])
    if ii > 10:
        break
# plt.show()
a = torch.tensor([[1,2,3],[1,2,3]]).float()
print(input_dict['adjacency'][0,0])
print(f1(input_dict['adjacency'], input_dict['num_agent_mask'])[0,0])

tensor([-0.0147, -0.0019,  0.0052, -0.0009,  0.0026, -0.0006,  0.0020, -0.0007,
         0.0001, -0.0009, -0.0002, -0.0030,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000])
tensor([2.1649, 0.9039, 1.1309, 0.8452, 0.9480, 0.8276, 0.9109, 0.8326, 0.8028,
        0.8466, 0.8075, 0.9792, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000])


In [20]:
from matplotlib import pyplot as plt

for ii, d in enumerate(val_dataset):
    
    input_dict, traget_dict = d[0], d[1]
#     for i in range(100):
#         xy = d[0]['agent_features'][i]
#         xy.shape
#         plt.scatter(xy[:,0], xy[:, 1])
    if ii > 10:
        break
# plt.show()


In [44]:
def f2(w, mask):
    for i in range(len(input_dict["adjacency"])):
        num_agent = int(torch.sum(mask[0]))
        w[0,0, :num_agent] = to_gaussian(w[0,0, :num_agent], mean = 1, std = 0.000001)
    return w

f2(input_dict['adjacency'], input_dict['num_agent_mask'])[0][0]

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000])